# GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b66bdc0d-a59b-775a-b617-26863b17dfc5)


# Drive Mount

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# MMCV install

In [3]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.1/index.html
!git clone https://github.com/open-mmlab/mmpose.git

Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.1/index.html
     |████████████████████████████████| 317kB 18.4MB/s 
     |████████████████████████████████| 194kB 42.7MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.9-cp37-cp37m-linux_x86_64.whl size=29316912 sha256=9b1d33c172d872fd3c8053dc78f222a0194bb5b6e326c44c3df9a09900f66362
  Stored in directory: /root/.cache/pip/wheels/e0/2b/d5/410850bd0f29e241507daf354083142a0c25d66667f22e707a
Successfully built mmcv-full
Cloning into 'mmpose'...
remote: Enumerating objects: 11215, done.
remote: Counting objects: 100% (3107/3107), done.
remote: Compressing objects: 100% (1139/1139), done.
remote: Total 11215 (delta 2203), reused 2490 (delta 1925), pack-reused 8108
Receiving objects: 100% (11215/11215), 20.76 MiB | 40.12 MiB/s, done.
Resolving deltas: 100% (7436/7436), done.


# MMPOSE install

In [4]:
%cd mmpose/
!pip install -r requirements.txt
!python setup.py develop

/content/mmpose
Ignoring dataclasses: markers 'python_version == "3.6"' don't match your environment
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 317kB 31.5MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Using cached https://files.pythonhosted.org/packages/f4/f5/6605d73bf3f4c198915872111b10c4b3c2dccd8485f47b7290ceef037190/pytest_runner-5.3.1-py3-none-any.whl
     |████████████████████████████████| 122kB 48.5MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 12.2MB 257kB/s 
     |████████████████████████████████| 4.5MB 42.6MB/s 
  Cloning https://github.com/svenkreiss/poseval.git to /tmp/pip-install-p4gwqds_/poseval
  Running command git clone -q https://github.com/svenkreiss/poseval.git /tmp/pip-install-p4gwqds_/poseval
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 

# numpy & opencv update

In [5]:
!pip uninstall numpy -y
!pip uninstall opencv-python -y
!pip install numpy
!pip install --upgrade pip
!pip install opencv-python==4.0.0.21

Uninstalling numpy-1.19.5:
  Successfully uninstalled numpy-1.19.5
Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 15.7MB 190kB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: mmpose 0.16.0 requires opencv-python, which is not installed.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 1.6MB 30.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 25.4 MB 84 kB/s 


# ^^ **restart the runtime** ^^

# Imports

In [1]:
import numpy as np
import shutil, os
import cv2

from tqdm import tqdm

from argparse import ArgumentParser,Namespace

from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result)

from matplotlib import image
from matplotlib import pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = [10,10]

# Important Directories

In [2]:
#################################################################
signer = '2'
#################################################################
zip_file = f'/content/drive/.shortcut-targets-by-id/10MbzXPgqUxsh3avgtcKxDVUMb4tAKl-k/Keypoint to GCN/Dataset/AUTSL_zipped/train/signer{signer}.zip'
unzip_directory = '/content/train_vid/'
output_directory = '/content/drive/.shortcut-targets-by-id/10MbzXPgqUxsh3avgtcKxDVUMb4tAKl-k/Keypoint to GCN/Dataset/train_npy_files_Darkpose/'

## extra depth for unzipped file
all_files_dirctory = unzip_directory 
all_files_dirctory

'/content/train_vid/'

# Unzipping

In [3]:
shutil.unpack_archive(zip_file,unzip_directory)

# Counting Files

In [4]:
ls = os.listdir(all_files_dirctory)
ls.sort()
len(ls)

2712

# Counting Signers

In [5]:
dc = {}
for l in ls:
    t = l.split('_')
    if t[2] != 'depth.mp4':
        if t[0] not in dc.keys():
            dc[t[0]] = 0
        dc[t[0]] += 1

for k in dc.keys():
    print(f"{k} \t: {dc[k]}")
print(f'Total {len(dc)} signer')

signer2 	: 1356
Total 1 signer


# Removing depth file and other signers

In [6]:
ls = os.listdir(all_files_dirctory)
ls.sort()

for l in ls:
    parts = l.split('_')
    if 'depth.mp4' in parts:
        pat = os.path.join(all_files_dirctory,l)
        #print(pat)
        os.remove(pat)


ls = os.listdir(all_files_dirctory)
ls.sort()
len(ls)

1356

In [7]:
%cd /content
!pwd
ls[0]

/content
/content


'signer2_sample1000_color.mp4'

# Already done

In [12]:
ts = os.listdir(output_directory)
f'{ls[0][:-4]}.npy' in ts

True

# Main Function

In [13]:
def my_main(video_path,output_path):
    #args = parser.parse_args()
    args = Namespace(device='cuda:0',
                     kpt_thr=0.3, 
                     pose_checkpoint='https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth', 
                     pose_config='mmpose/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/hrnet_w48_coco_wholebody_384x288_dark_plus.py', 
                     video_path=video_path)

    #assert args.show or (args.out_video_root != '')
    # build the pose model from a config file and a checkpoint file
    pose_model = init_pose_model(
        args.pose_config, args.pose_checkpoint, device=args.device.lower())

    dataset = pose_model.cfg.data['test']['type']

    cap = cv2.VideoCapture(args.video_path)
    assert cap.isOpened(), f'Faild to load video file {args.video_path}'

    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    

    # optional
    return_heatmap = False

    # e.g. use ('backbone', ) to return backbone feature
    output_layer_names = None

    
    output_list = []
    while (cap.isOpened()):
        flag, img = cap.read()
        if not flag:
            break

        img = cv2.flip(img, flipCode=1)

        # keep the person class bounding boxes.
        person_results = [{'bbox': np.array([0, 0, size[0], size[1]])}]

        # test a single image, with a list of bboxes.
        pose_results, returned_outputs = inference_top_down_pose_model(
            pose_model,
            img,
            person_results,
            format='xyxy',
            dataset=dataset,
            return_heatmap=return_heatmap,
            outputs=output_layer_names)

        output_list.append(pose_results[0]['keypoints'])

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        

    cap.release()
    cv2.destroyAllWindows()
    output_list = np.array(output_list)
    np.save(output_path, output_list)
    


for l in tqdm(ls):
    npy_file = f'{l[:-4]}.npy'
    vp = os.path.join(all_files_dirctory,l)
    op = os.path.join(output_directory,npy_file)
    if npy_file not in ts:
        print(l[:-4],end = ' ->')
        my_main(vp,op)

 59%|█████▊    | 796/1356 [00:00<00:00, 3659.79it/s]

signer2_sample89_color ->Use load_from_http loader


Downloading: "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth" to /root/.cache/torch/hub/checkpoints/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth


 59%|█████▊    | 796/1356 [00:19<00:00, 3659.79it/s]

 92%|█████████▏| 1244/1356 [00:53<00:04, 27.81it/s] 

...

100%|█████████▉| 1354/1356 [21:02<00:22, 11.27s/it]

signer2_sample99_color ->Use load_from_http loader


100%|█████████▉| 1355/1356 [21:12<00:10, 10.91s/it]

signer2_sample9_color ->Use load_from_http loader


100%|██████████| 1356/1356 [21:24<00:00,  1.06it/s]


In [14]:
2+2

4

In [15]:
len(ts)

15502